In [1]:
import numpy as np
import pandas as pd
import random
import datetime as dt
import seaborn as sns
from re import search

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

random.seed(333)
pd.options.mode.chained_assignment = None
import os

from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor

In [2]:
auto_base = pd.read_csv('./working/AUTOMOTIVE.csv')
auto_original = pd.read_csv('./original/AUTOMOTIVE.csv')

In [3]:
auto_base.sort_values('id').head(200)

,date,id,sales,day_of_week,weekend,month,day_of_year,week_of_year,year,national_day,regional_day,local_day,onpromotion,dcoilwtico
0,2013-01-01,0,0.693147,2,0,1,1,1,2013,1.0,0.0,0.0,0,0.00
1,2013-01-01,33,0.693147,2,0,1,1,1,2013,1.0,0.0,0.0,0,0.00
18,2013-01-01,66,0.693147,2,0,1,1,1,2013,1.0,0.0,0.0,0,0.00
19,2013-01-01,99,0.693147,2,0,1,1,1,2013,1.0,0.0,0.0,0,0.00
20,2013-01-01,132,0.693147,2,0,1,1,1,2013,1.0,0.0,0.0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2013-01-04,6435,2.197225,5,0,1,4,1,2013,0.0,0.0,0.0,0,93.12
210,2013-01-04,6468,1.609438,5,0,1,4,1,2013,0.0,0.0,0.0,0,93.12
211,2013-01-04,6501,1.098612,5,0,1,4,1,2013,0.0,0.0,0.0,0,93.12
208,2013-01-04,6534,0.693147,5,0,1,4,1,2013,0.0,0.0,0.0,0,93.12


In [4]:
auto_original.sort_values('id').head(200)

,id,date,store_nbr,sales,onpromotion,Payday,Payday_Scale,DayOfTheYear,Year,dayoftheweek_scale,...,City_Density_Category3,City_Density_Category2,City_Density_Category5,City_Size_Category2,City_Size_Category0,City_Size_Category1,City_Size_Category3,earthquake_day,earthquake_impact,school_preparation
0,0,2013-01-01,1,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
1,33,2013-01-01,10,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
2,66,2013-01-01,11,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
3,99,2013-01-01,12,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
4,132,2013-01-01,13,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6435,2013-01-04,4,2.197225,0,0.0,4,4,2013,5,...,0,0,0,1,0,0,0,0,0,0
196,6468,2013-01-04,40,1.609438,0,0.0,4,4,2013,5,...,0,0,0,0,1,0,0,0,0,0
197,6501,2013-01-04,41,1.098612,0,0.0,4,4,2013,5,...,0,0,0,0,1,0,0,0,0,0
198,6534,2013-01-04,42,0.693147,0,0.0,4,4,2013,5,...,0,0,0,0,1,0,0,0,0,0


In [5]:
auto_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91800 entries, 0 to 91799
Data columns (total 146 columns):
 #    Column                     Dtype  
---   ------                     -----  
 0    id                         int64  
 1    date                       object 
 2    store_nbr                  int64  
 3    sales                      float64
 4    onpromotion                int64  
 5    Payday                     float64
 6    Payday_Scale               int64  
 7    DayOfTheYear               int64  
 8    Year                       int64  
 9    dayoftheweek_scale         int64  
 10   DayOfTheWeekTuesday        int64  
 11   DayOfTheWeekWednesday      int64  
 12   DayOfTheWeekThursday       int64  
 13   DayOfTheWeekFriday         int64  
 14   DayOfTheWeekSaturday       int64  
 15   DayOfTheWeekSunday         int64  
 16   DayOfTheWeekMonday         int64  
 17   MonthOfTheYearJanuary      int64  
 18   MonthOfTheYearFebruary     int64  
 19   MonthOfTheYearMarch    

In [6]:
auto_original

,id,date,store_nbr,sales,onpromotion,Payday,Payday_Scale,DayOfTheYear,Year,dayoftheweek_scale,...,City_Density_Category3,City_Density_Category2,City_Density_Category5,City_Size_Category2,City_Size_Category0,City_Size_Category1,City_Size_Category3,earthquake_day,earthquake_impact,school_preparation
0,0,2013-01-01,1,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
1,33,2013-01-01,10,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
2,66,2013-01-01,11,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
3,99,2013-01-01,12,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
4,132,2013-01-01,13,0.693147,0,0.0,1,1,2013,2,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91795,3029235,2017-08-31,54,NaN,0,0.0,16,243,2017,4,...,0,0,0,0,0,0,1,0,0,0
91796,3029268,2017-08-31,6,NaN,0,0.0,16,243,2017,4,...,0,0,0,1,0,0,0,0,0,0
91797,3029301,2017-08-31,7,NaN,0,0.0,16,243,2017,4,...,0,0,0,1,0,0,0,0,0,0
91798,3029334,2017-08-31,8,NaN,0,0.0,16,243,2017,4,...,0,0,0,1,0,0,0,0,0,0


In [7]:
auto_original_test = auto_original.iloc[-864:]

In [8]:
auto_original_test

,id,date,store_nbr,sales,onpromotion,Payday,Payday_Scale,DayOfTheYear,Year,dayoftheweek_scale,...,City_Density_Category3,City_Density_Category2,City_Density_Category5,City_Size_Category2,City_Size_Category0,City_Size_Category1,City_Size_Category3,earthquake_day,earthquake_impact,school_preparation
90936,3000888,2017-08-16,1,NaN,0,0.0,1,228,2017,3,...,0,0,0,1,0,0,0,0,0,0
90937,3000921,2017-08-16,10,NaN,0,0.0,1,228,2017,3,...,0,0,0,1,0,0,0,0,0,0
90938,3000954,2017-08-16,11,NaN,0,0.0,1,228,2017,3,...,0,0,0,1,0,0,0,0,0,0
90939,3000987,2017-08-16,12,NaN,0,0.0,1,228,2017,3,...,0,0,0,1,0,0,0,0,0,0
90940,3001020,2017-08-16,13,NaN,0,0.0,1,228,2017,3,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91795,3029235,2017-08-31,54,NaN,0,0.0,16,243,2017,4,...,0,0,0,0,0,0,1,0,0,0
91796,3029268,2017-08-31,6,NaN,0,0.0,16,243,2017,4,...,0,0,0,1,0,0,0,0,0,0
91797,3029301,2017-08-31,7,NaN,0,0.0,16,243,2017,4,...,0,0,0,1,0,0,0,0,0,0
91798,3029334,2017-08-31,8,NaN,0,0.0,16,243,2017,4,...,0,0,0,1,0,0,0,0,0,0


In [6]:
test = pd.read_csv('test_features.csv', parse_dates=['date'])
train = pd.read_csv('train_features.csv', parse_dates=['date'])

In [7]:
train_test = pd.concat([train, test], ignore_index=True)

In [8]:
train_test.dtypes

id                       int64
date            datetime64[ns]
store_nbr                int64
family                  object
sales                  float64
onpromotion              int64
day_of_week              int64
weekend                  int64
month                    int64
day_of_year              int64
week_of_year             int64
year                     int64
dcoilwtico             float64
city                    object
state                   object
type                    object
cluster                  int64
transactions           float64
national_day           float64
regional_day           float64
local_day              float64
dtype: object

In [9]:
def create_multi_store_one_product_df(df, product_name):

    multistore_single_product = df[df['family'] == product_name]

    return multistore_single_product

In [20]:
def full_product_pipeline(family):

    originaltrainFull = pd.read_csv('train_features.csv')
    originaltest = pd.read_csv('test_features.csv')

    multistore_product = create_multi_store_one_product_df(originaltrainFull, family)

    # merge with test:
    multistore_product_test = create_multi_store_one_product_df(originaltest, family)
    multistore_product_test['sales'] = np.nan

    del originaltrainFull
    del originaltest

    # take log of sales:
    multistore_product['sales'] = np.log1p(multistore_product['sales']+1)

    msp_full = pd.concat([multistore_product, multistore_product_test])

    # reset index:
    msp_full.reset_index(inplace=True, drop=True)



    
    # Convert objects to numbers 
    
    #msp_full = pd.get_dummies(data=msp_full, columns=['store_nbr','cluster','type'], dtype=np.int64)
        

    # Clean DF before modelling 
    

    msp_full.drop(['family'], axis=1, inplace=True)
    #msp_full.drop(['city','state','onpromotion'], axis=1, inplace=True)
    
    msp_full = msp_full[['date','id','sales','day_of_week','weekend','month','day_of_year','week_of_year','year',
                         'national_day','regional_day','local_day','onpromotion','dcoilwtico']]

    return msp_full

In [21]:
# --- Execute Full Product Pipeline for each product --- #

# List all product families:

list_of_families = ['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
                    'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
                    'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
                    'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
                    'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
                    'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
                    'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
                    'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
                    'SEAFOOD']

# Create new .csv for each product family:

for x in list_of_families:

    this_df = full_product_pipeline(x)

    if x == 'BREAD/BAKERY':

            x = 'BREADBAKERY'

    print('Completed eda for ' + str(x))
    this_df.to_csv('./working/'+str(x)+'.csv', index=False)

Completed eda for AUTOMOTIVE
Completed eda for BABY CARE
Completed eda for BEAUTY
Completed eda for BEVERAGES
Completed eda for BOOKS
Completed eda for BREADBAKERY
Completed eda for CELEBRATION
Completed eda for CLEANING
Completed eda for DAIRY
Completed eda for DELI
Completed eda for EGGS
Completed eda for FROZEN FOODS
Completed eda for GROCERY I
Completed eda for GROCERY II
Completed eda for HARDWARE
Completed eda for HOME AND KITCHEN I
Completed eda for HOME AND KITCHEN II
Completed eda for HOME APPLIANCES
Completed eda for HOME CARE
Completed eda for LADIESWEAR
Completed eda for LAWN AND GARDEN
Completed eda for LINGERIE
Completed eda for LIQUOR,WINE,BEER
Completed eda for MAGAZINES
Completed eda for MEATS
Completed eda for PERSONAL CARE
Completed eda for PET SUPPLIES
Completed eda for PLAYERS AND ELECTRONICS
Completed eda for POULTRY
Completed eda for PREPARED FOODS
Completed eda for PRODUCE
Completed eda for SCHOOL AND OFFICE SUPPLIES
Completed eda for SEAFOOD


In [22]:
sample_submission = pd.read_csv('sample_submission.csv')

In [23]:
def scorethis_rmsle(prediction_list, y_list):

    scorelist = list()

    for x in range(prediction_list.__len__()):


        log_score_x = np.abs(np.abs(prediction_list[x]) - np.abs(y_list[x]))
        
        try:
            [scorelist.append(y) for y in log_score_x.values]
        except:
            scorelist.append(log_score_x)

    score_array = np.array(scorelist)

    rmsle = np.sqrt(np.mean(score_array**2)) # sqrt of mean of power of difference of the logs
    rmsle = np.round(rmsle, 3)

    return rmsle

In [24]:
def create_validation(this_family_df, validation=True):
    
    if validation is True:
    
        this_family_df = this_family_df[:-864]
        # Remove the 864 top submission rows if it is for validation
    
    this_family_sales = this_family_df['sales']

    this_family_df.drop(['sales', 'date'], axis=1, inplace=True)

    ########################
    # Scale Data           #
    ########################

    scaler = MinMaxScaler()
    this_family_df[this_family_df.columns] = scaler.fit_transform(this_family_df[this_family_df.columns])

    ########################
    # Split Train and Test #
    ########################

    test = this_family_df.iloc[-864:]
    test_y = this_family_sales.iloc[-864:]

    train = this_family_df.iloc[:-864]
    train_y = this_family_sales.iloc[:-864]

    return train, train_y, test, test_y

In [25]:
def lgbmr_run(train, train_y, test, test_y,
           validation=True):
    
    #################
    # Create Model  #
    #################

    lgbmr_model = LGBMRegressor(
        colsample_bytree=0.7,
        learning_rate=0.055,
        min_child_samples=10,
        num_leaves=19,
        objective='regression',
        n_estimators=1000,
        n_jobs=4,
        random_state=337)

    #################
    # Execute LGBMR #
    #################

    lgbmr_model.fit(train, train_y)
    lgbmr_pred = lgbmr_model.predict(test).tolist()
    lgbmr_pred = [round(x, 2) for x in lgbmr_pred]
    
    if validation == True:
        
        # validation set also has ground truths:
        test_y = test_y.to_list()

        return lgbmr_pred, test_y

    else:

        return lgbmr_pred

In [26]:
def execute_validation(thisfunc):

    double_list_of_predictions = []
    double_list_of_ground_truths = []

    for x in list_of_families: # 33
        
        if x == 'BREAD/BAKERY':

            x = 'BREADBAKERY'
            # Otherwise would create an error searching for the BREAD/ directory instead of the file

        print('Evaluating '+str(x)+'...')
        
        this_df = pd.read_csv('./working/' + str(x) + '.csv')

        train, train_y, test, test_y = create_validation(this_df)
        pred, y = thisfunc(train, train_y, test, test_y, validation=True)
        
        if x == 'BOOKS':

            zero_list = []

            for g in range(864):

                zero_list.append(0.6931471805599453) 
                # this will be exactly 0 when we transform our predictions again
                # to before we did log(sales +1)

            double_list_of_predictions.append(zero_list)
            double_list_of_ground_truths.append(y) 
            
        else:
            
            double_list_of_predictions.append(pred) # 33 * [864]
            double_list_of_ground_truths.append(y) # 33 * [864]

    list_of_predictions = list()
    list_of_ground_truths = list()

    for x in double_list_of_predictions:
        for y in x:
            list_of_predictions.append(y) # unpack 33 * 864

    for x in double_list_of_ground_truths:
        for z in x:
            list_of_ground_truths.append(z) # unpack 33 * 864

    return list_of_predictions, list_of_ground_truths

In [27]:
# --- Execute LGBMR Model On Validation Set --- #

# Run this code if you want to do a validation test + see the score:

list_of_lgbmr_predictions, list_of_ground_truths = execute_validation(lgbmr_run)
scorethis_rmsle(list_of_lgbmr_predictions, list_of_ground_truths)

Evaluating AUTOMOTIVE...
Evaluating BABY CARE...
Evaluating BEAUTY...
Evaluating BEVERAGES...
Evaluating BOOKS...
Evaluating BREADBAKERY...
Evaluating CELEBRATION...
Evaluating CLEANING...
Evaluating DAIRY...
Evaluating DELI...
Evaluating EGGS...
Evaluating FROZEN FOODS...
Evaluating GROCERY I...
Evaluating GROCERY II...
Evaluating HARDWARE...
Evaluating HOME AND KITCHEN I...
Evaluating HOME AND KITCHEN II...
Evaluating HOME APPLIANCES...
Evaluating HOME CARE...
Evaluating LADIESWEAR...
Evaluating LAWN AND GARDEN...
Evaluating LINGERIE...
Evaluating LIQUOR,WINE,BEER...
Evaluating MAGAZINES...
Evaluating MEATS...
Evaluating PERSONAL CARE...
Evaluating PET SUPPLIES...
Evaluating PLAYERS AND ELECTRONICS...
Evaluating POULTRY...
Evaluating PREPARED FOODS...
Evaluating PRODUCE...
Evaluating SCHOOL AND OFFICE SUPPLIES...
Evaluating SEAFOOD...


0.705

In [28]:
def execute_submission(thisfunc):

    list_of_predictions = []

    for x in list_of_families:
        
        if x == 'BREAD/BAKERY':

            x = 'BREADBAKERY'
            # Otherwise would create an error searching for the BREAD/ directory instead of the file

        print('Evaluating '+str(x)+'...')
        this_df = pd.read_csv('./working/' + str(x) + '.csv')
        
        if x == 'BOOKS':

            zero_list = []

            for g in range(864):

                zero_list.append(0.6931471805599453) 
                # this will be exactly 0 when we transform our predictions again
                # to before we did log(sales +1)

            list_of_predictions.append(zero_list)

        else:
    
            train, train_y, test, test_y = create_validation(this_df, validation=False)
            pred = thisfunc(train, train_y, test, test_y=None, validation=False)
            list_of_predictions.append(pred)
    
    ###############################
    # Put Back In Submission Form # 
    ###############################
    
    restructured_predictions = list()

    for y in range(864):

        for z in range(33):
            restructured_predictions.append(list_of_predictions[z][y])

    restructured_predictions = np.expm1(restructured_predictions) - 1

    return restructured_predictions

In [29]:
# --- Execute Submission --- #

restructured_predictions = execute_submission(lgbmr_run)
sample_submission['sales'] = restructured_predictions

# Convert some (slightly) negative predictions to a zero prediction:
sample_submission['sales'] = [0 if x < 0 else x for x in sample_submission['sales']]

sample_submission.to_csv('./submissions/submit-33-baseline.csv', index=False)

Evaluating AUTOMOTIVE...
Evaluating BABY CARE...
Evaluating BEAUTY...
Evaluating BEVERAGES...
Evaluating BOOKS...
Evaluating BREADBAKERY...
Evaluating CELEBRATION...
Evaluating CLEANING...
Evaluating DAIRY...
Evaluating DELI...
Evaluating EGGS...
Evaluating FROZEN FOODS...
Evaluating GROCERY I...
Evaluating GROCERY II...
Evaluating HARDWARE...
Evaluating HOME AND KITCHEN I...
Evaluating HOME AND KITCHEN II...
Evaluating HOME APPLIANCES...
Evaluating HOME CARE...
Evaluating LADIESWEAR...
Evaluating LAWN AND GARDEN...
Evaluating LINGERIE...
Evaluating LIQUOR,WINE,BEER...
Evaluating MAGAZINES...
Evaluating MEATS...
Evaluating PERSONAL CARE...
Evaluating PET SUPPLIES...
Evaluating PLAYERS AND ELECTRONICS...
Evaluating POULTRY...
Evaluating PREPARED FOODS...
Evaluating PRODUCE...
Evaluating SCHOOL AND OFFICE SUPPLIES...
Evaluating SEAFOOD...


The original model from the Kaggle notebook had a score of 0.337 and a Kaggle score of 0.40213. This initial data had a score of 0.333 and a Kaggle score of 2.48.

Taking out city, state and type created score of 0.331 and Kaggle score of 2.75. Using one hot encoding for city, state and type created a score of 0.332 and a Kaggle score of 2.50. 

Just using sales as the only feature had a score of 0.811 and a Kaggle score of 0.90548. Adding back the date variables had a score of 0.801 and a Kaggle score of 0.89538. Then adding back in the holidays variables had a score of .803 and a Kaggle score of 0.89344. 

Adding back in store_nbr, onpromotion and dcoilwtico created a score of 0.352 and a Kaggle score of 1.22685. Then taking oil out for a score of 0.348 and a Kaggle score of 1.23389. Adding back oil and taking out onpromotion created a score of 0.403. Adding back onpromotion and taking out store created a score of 0.705 and a Kaggle score of 1.0568.

Including full dataset and setting city, state, type to dummies with dtype int64 created a score of 0.332. Including full dataset except city and state, with dummies for store_nbr, cluster and type created a score of 0.332. Taking out onpromotion created a score of 0.391 and a Kaggle score of 3.7039.